In [17]:
from monostable_model import ion_channel_model, model_force_square, random_force_gauss
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, fixed, SelectionSlider


def ion_channel_interactive(
    a, closed_0, closed_1, opened_0, opened_1, D, delta_t, records, model_force, random_force
):
    closed = (closed_0, closed_1)
    opened = (opened_0, opened_1)
    ion_channel_model(a, closed, opened, D, delta_t, records, model_force, random_force)

# Interact with the function
interact(
    ion_channel_interactive,
    a=FloatSlider(min=0.1, max=10.0, step=0.1, value=1, description='a'),
    closed_0=IntSlider(min=-10, max=10, step=1, value=-1, description='Closed value'),
    closed_1=IntSlider(min=1000, max=10000, step=500, value=5000, description='Closed time scale'),
    opened_0=IntSlider(min=-10, max=10, step=1, value=1, description='Opened value'),
    opened_1=IntSlider(min=1000, max=10000, step=500, value=2500, description='Opened time scale'),
    D=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='D'),
    delta_t=SelectionSlider(
        options=[10**-i for i in range(1, 5)],
        value=0.01,
        description='Delta t',
    ),
    records=IntSlider(min=1000, max=100000, step=1000, value=50000, description='Records'),
    model_force=Dropdown(
        options={'Square Force': model_force_square, 'Other Force': lambda x: x**2},
        value=model_force_square,
        description='Model Force',
    ),
    random_force=fixed(random_force_gauss),  # Keep it fixed for simplicity
)

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, min=0.1), IntSlider(value=-1, descript…

<function __main__.ion_channel_interactive(a, closed_0, closed_1, opened_0, opened_1, D, delta_t, records, model_force, random_force)>